In [1]:
import os
import pandas as pd
from Bio import SeqIO
import numpy as np
import gffutils
from collections import Counter

### Unzip db builds (if necessary):

    tar -zxf nextflow_db.tar.gz
    tar -zxf midasdb_localdb.tar.gz

### Output of nextflow pipeline

In [2]:
my_db_path = '/wynton/protected/home/sirota/clairedubin/MIDAS3_nextflow/testing/nextflow_db'

### Output run generated Chunyu using original MIDAS scripts

Original code here: https://github.com/czbiohub-sf/MIDAS/blob/master/tests/test_db.sh

In [3]:
correct_db_path = '/wynton/protected/home/sirota/clairedubin/MIDAS3_nextflow/testing/midasdb_localdb'

### Comparing folders

Mine is missing genomes.tsv, markers_models, and cleaned_imports because the nextflow pipeline doesn't require these input files to be stored within the database directory. 

In [4]:
!ls {my_db_path}

chunks	gene_annotations  markers  pangenomes  pangenomes_annotation


In [5]:
!ls {correct_db_path}

chunks		 gene_annotations  markers	   pangenomes
cleaned_imports  genomes.tsv	   markers_models  pangenomes_annotation


### Get list of files in each database

In [6]:
db = 'nextflow_db'
my_db_files = []

for path, subdirs, files in os.walk(db):
    for name in files:
        
        if name == 'genomes.tsv' or 'cleaned_imports' in path:
            continue
        
        my_db_files += [os.path.join(path, name).replace(db+'/', '')]

In [7]:
db = 'midasdb_localdb'
correct_db_files = []

for path, subdirs, files in os.walk(db):
    for name in files:
        if '.log' in name: #ignore missing log files
            continue
        if name == 'genomes.tsv' or 'cleaned_imports' in path or 'markers_models' in path:
            continue
        correct_db_files += [os.path.join(path, name).replace(db+'/', '')]

### Compare list of files generated by each database

 

In [8]:
not_matched = []

for f in correct_db_files:
    
    if f in my_db_files:
        continue
            
    file_only = f.split('/')[-1]
    
    if '.log' in f:
        continue
    
    potential_files = [i for i in my_db_files if i.split('/')[-1] == file_only]
    if len(potential_files) > 0:

        
        if '117086' in f:
            potential_files = [i for i in potential_files if '117086' in i]
        elif '117088' in f:
            potential_files = [i for i in potential_files if '117088' in i]
        
        print(f)
        print('possible matches:')
        for i in potential_files:
            print(i)
    
    else:
        print(f)
        print('no matches found')
    
    print()
    
    not_matched+=[f]
        

pangenomes/117088/temp/vsearch/genes.ffn
possible matches:
pangenomes/117088/genes.ffn
pangenomes/117088/temp/cdhit/genes.ffn

pangenomes/117086/temp/vsearch/genes.ffn
possible matches:
pangenomes/117086/genes.ffn
pangenomes/117086/temp/cdhit/genes.ffn



### Unmatched files:
     
     'pangenomes/117086/temp/vsearch/genes.ffn' - can ignore, this is the same as nextflow_db/pangenomes/117086/genes.ffn, just a different path

In [9]:
not_matched

['pangenomes/117088/temp/vsearch/genes.ffn',
 'pangenomes/117086/temp/vsearch/genes.ffn']

In [10]:
#genes.ffn files are the same
!diff <(sort midasdb_localdb/pangenomes/117086/temp/vsearch/genes.ffn) <(sort nextflow_db/pangenomes/117086/genes.ffn)
!diff <(sort midasdb_localdb/pangenomes/117088/temp/vsearch/genes.ffn) <(sort nextflow_db/pangenomes/117088/genes.ffn)


### Are the contents of each file the same?

In [11]:
#ignore lines with these words
diff_exclude = '" -I "'.join(['-I "clairedubin','pollard', #names in most paths
                        '^\#.*', #exclude comment lines  
                        'time', 'tm_sec', 'tm_min', 'tm_hour', 'tm_mday',
                        'tm_mon', 'tm_wday', 'tm_yday', 'Date', #timestamp stuff
                        'Database name', #path to databases (some aren't full paths)
                        '\<Statistics_',  #comments in resfinder outputs
                             
                             ])
diff_exclude += '"'

In [12]:
#these will cause diff to fail
binary_files = ['markers/phyeco/phyeco.fa.bwt',
               'markers/phyeco/phyeco.fa.sa']

In [13]:
different_contents = {}
same_contents = []
different_but_ok_files = []

for f in correct_db_files:
        
    if f in not_matched or f in binary_files:
        continue
        
    if 'temp' in f or 'tmp' in f:#skip intermediate files
        continue
        
    #check if contents are the same, excluding words specified above and sorting lines
    cmd = f"diff {diff_exclude} <(sort nextflow_db/{f}) <(sort midasdb_localdb/{f})"
    diff_out = !{cmd}
    
    if len(diff_out) == 0:
        same_contents += [f]
        continue
    
    different_contents[f] =  [f'nextflow_db/{f}', f'midasdb_localdb/{f}', diff_out]
        

In [14]:
len(same_contents), len(different_contents)

(344, 36)

### Which subfolders have differing files?

In [15]:
subs = []

for i in different_contents.keys():
    
    subfolder = i.split('/')[0]
    subs += [subfolder]
    
Counter(subs)


Counter({'pangenomes_annotation': 31, 'gene_annotations': 4, 'markers': 1})

### Compare differing files in gene_annotations directory

These are the same - diff just can't handle the DB files

In [16]:
g = []
for i in different_contents.keys():
    if 'gene_annotations/' in i:
        print(i)
        g += [i]

gene_annotations/117086/GCF_900752885.1/GCF_900752885.1.gff.db
gene_annotations/117086/GCA_900552055.1/GCA_900552055.1.gff.db
gene_annotations/117088/GCA_007120565.1/GCA_007120565.1.gff.db
gene_annotations/117088/GCA_007135585.1/GCA_007135585.1.gff.db


In [17]:
for i in g:

    test_gff = gffutils.interface.FeatureDB(f'nextflow_db/{i}')
    correct_gff = gffutils.interface.FeatureDB(f'midasdb_localdb/{i}')

    c = correct_gff.all_features()

    for i in test_gff.all_features():
        if i != next(c):
            print(i)

In [18]:
ignore_diffs = []
ignore_diffs += [i for i in different_contents.keys() if 'gene_annotations/' in i]

### Compare differing files in markers directory

Header files are just different because of the date/path in the header.

In [19]:
for i in different_contents.keys():
    if 'markers/' in i:
        print(i)

markers/phyeco/phyeco.fa.header


In [20]:
f = 'markers/phyeco/phyeco.fa'
cmd = f"diff {diff_exclude} <(sort nextflow_db/{f}) <(sort midasdb_localdb/{f})"
!{cmd}

In [21]:
f = 'markers/phyeco/phyeco.fa.header'
cmd = f"diff {diff_exclude} <(sort nextflow_db/{f}) <(sort midasdb_localdb/{f})"
!{cmd}

92,99c92,99
< 	tm_hour=11
< 	tm_mday=6
< 	tm_min=18
< 	tm_mon=1
< 	tm_sec=14
< 	tm_wday=4
< 	tm_yday=36
< 	tm_year=125
---
> 	tm_hour=14
> 	tm_mday=24
> 	tm_min=26
> 	tm_mon=8
> 	tm_sec=46
> 	tm_wday=2
> 	tm_yday=267
> 	tm_year=124


In [22]:
ignore_diffs += ['markers/phyeco/phyeco.fa.header']

## Compare differing files in pangenomes_annotation directory


In [23]:
for i in different_contents.keys():
    if 'pangenomes_annotation/' in i:
        print(i)

pangenomes_annotation/01_mge/117088/GCA_007135585.1/genomad_output/GCA_007135585.1_aggregated_classification/GCA_007135585.1_aggregated_classification.npz
pangenomes_annotation/01_mge/117088/GCA_007135585.1/genomad_output/GCA_007135585.1_aggregated_classification/GCA_007135585.1_aggregated_classification.tsv
pangenomes_annotation/01_mge/117088/GCA_007135585.1/genomad_output/GCA_007135585.1_marker_classification/GCA_007135585.1_features.npz
pangenomes_annotation/01_mge/117088/GCA_007135585.1/genomad_output/GCA_007135585.1_score_calibration/GCA_007135585.1_calibrated_aggregated_classification.tsv
pangenomes_annotation/01_mge/117088/GCA_007135585.1/genomad_output/GCA_007135585.1_score_calibration/GCA_007135585.1_calibrated_nn_classification.tsv
pangenomes_annotation/01_mge/117088/GCA_007135585.1/genomad_output/GCA_007135585.1_score_calibration/GCA_007135585.1_calibrated_nn_classification.npz
pangenomes_annotation/01_mge/117088/GCA_007135585.1/genomad_output/GCA_007135585.1_score_calibrati

### Genomad outputs

TSVs and .npz files are the same, except for a <0.001 rounding difference in a few scores. 



In [24]:
genomad_tsvs = [i for i in different_contents.keys() if 'genomad_output' in i and i.endswith('.tsv')]

In [25]:
for f in genomad_tsvs:

    df1 = pd.read_csv(f'nextflow_db/{f}', sep='\t', index_col=0)
    df2 = pd.read_csv(f'midasdb_localdb/{f}', sep='\t', index_col=0)


    for gene, row1 in df1.iterrows():

        row2 = df2.loc[gene]
        if row1.equals(row2):
            continue

        #check if it's +/- 0.0001 difference
        elif abs(row1-row2).sum() <= 0.001:
            continue

        else:
            print(gene)
            print(row1.values)
            print(row2.values)
            print(abs(row1-row2).sum())
            print()

In [26]:
npz_files = []

for f, (test_path, correct_path, diff_out) in different_contents.items():

    if '.npz' in f and 'genomad' in f:
        npz_files += [(test_path, correct_path)]
        continue

for (f1_path, f2_path) in npz_files:
    
    f1 = np.load(f1_path)
    f2 = np.load(f2_path)
    for i in f1:

        if (f1[i] == f2[i]).all():
            continue

        else:
            diffs = np.where(abs(f1[i] - f2[i]) > 0.001)
            if len(diffs) != 0:
                print(diffs)


(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int

In [27]:
[i for i in different_contents.keys() if 'pangenomes_annotation/' in i and 'genomad' in i and not ('.npz' in i or '.tsv' in i)]

[]

In [28]:
ignore_diffs += [i for i in different_contents.keys() if 'pangenomes_annotation/' in i and 'genomad' in i and ('.npz' in i or '.tsv' in i)]

In [29]:
[i for i in different_contents.keys() if i not in ignore_diffs]

[]